In [53]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score,confusion_matrix,accuracy_score
from sklearn import tree


In [54]:
df = pd.read_excel("loan.xlsx")
df.head()

,Sex,Age,Time_at_address,Res_status,Telephone,Occupation,Job_status,Time_employed,Time_bank,Liab_ref,Acc_ref,Home_Expn,Balance,Decision
0,M,50.750000,0.585,owner,given,unemploye,unemploye,0,0,f,given,145,0,reject
1,M,19.670000,10.000,rent,not_given,labourer,governmen,0,0,t,given,140,0,reject
2,F,52.830002,15.000,owner,given,creative_,private_s,5,14,f,given,0,2200,accept
3,M,22.670000,2.540,rent,not_given,creative_,governmen,2,0,f,given,0,0,accept
4,M,29.250000,13.000,owner,given,driver,governmen,0,0,f,given,228,0,reject


In [55]:
sex = pd.get_dummies(df['Sex'])
res_s = pd.get_dummies(df['Res_status'])
tele = pd.get_dummies(df['Telephone'])
occup= pd.get_dummies(df['Occupation'])
job_s = pd.get_dummies(df['Job_status'])
acc_r = pd.get_dummies(df['Acc_ref'])
lia_ref = pd.get_dummies(df['Liab_ref'])
dec = pd.get_dummies(df['Decision'])
df= pd.concat((df,sex,res_s,tele,occup,job_s,acc_r,lia_ref,dec),axis=1)

df.head(10)

,Sex,Age,Time_at_address,Res_status,Telephone,Occupation,Job_status,Time_employed,Time_bank,Liab_ref,...,retired,self_empl,student,unemploye,given,oth_inst_,f,t,accept,reject
0,M,50.750000,0.585,owner,given,unemploye,unemploye,0,0,f,...,0,0,0,1,1,0,1,0,0,1
1,M,19.670000,10.000,rent,not_given,labourer,governmen,0,0,t,...,0,0,0,0,1,0,0,1,0,1
2,F,52.830002,15.000,owner,given,creative_,private_s,5,14,f,...,0,0,0,0,1,0,1,0,1,0
3,M,22.670000,2.540,rent,not_given,creative_,governmen,2,0,f,...,0,0,0,0,1,0,1,0,1,0
4,M,29.250000,13.000,owner,given,driver,governmen,0,0,f,...,0,0,0,0,1,0,1,0,0,1
5,F,16.080000,0.335,owner,given,unemploye,unemploye,0,1,f,...,0,0,0,1,1,0,1,0,0,1
6,M,23.170000,11.125,owner,given,professio,governmen,0,1,f,...,0,0,0,0,1,0,1,0,1,0
7,F,27.580000,3.000,owner,given,manager,private_s,2,1,t,...,0,0,0,0,1,0,0,1,0,1
8,F,19.170000,5.415,owner,given,guard_etc,governmen,0,0,f,...,0,0,0,0,1,0,1,0,0,1
9,F,27.250000,0.290,owner,given,manager,governmen,0,1,t,...,0,0,0,0,1,0,0,1,0,1


In [56]:
df =df.drop(['Sex','Res_status','Telephone','Occupation','Job_status','Liab_ref','Acc_ref','Decision'],axis=1)

In [57]:
df

,Age,Time_at_address,Time_employed,Time_bank,Home_Expn,Balance,F,M,owner,rent,...,retired,self_empl,student,unemploye,given,oth_inst_,f,t,accept,reject
0,50.750000,0.585,0,0,145,0,0,1,1,0,...,0,0,0,1,1,0,1,0,0,1
1,19.670000,10.000,0,0,140,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,1
2,52.830002,15.000,5,14,0,2200,1,0,1,0,...,0,0,0,0,1,0,1,0,1,0
3,22.670000,2.540,2,0,0,0,0,1,0,1,...,0,0,0,0,1,0,1,0,1,0
4,29.250000,13.000,0,0,228,0,0,1,1,0,...,0,0,0,0,1,0,1,0,0,1
5,16.080000,0.335,0,1,160,126,1,0,1,0,...,0,0,0,1,1,0,1,0,0,1
6,23.170000,11.125,0,1,100,0,0,1,1,0,...,0,0,0,0,1,0,1,0,1,0
7,27.580000,3.000,2,1,280,10,1,0,1,0,...,0,0,0,0,1,0,0,1,0,1
8,19.170000,5.415,0,0,80,484,1,0,1,0,...,0,0,0,0,1,0,1,0,0,1
9,27.250000,0.290,0,1,272,108,1,0,1,0,...,0,0,0,0,1,0,0,1,0,1


In [58]:
X = df.values
y = df['accept'].values
X = df.drop(['accept','reject'],axis=1)

In [59]:
X

,Age,Time_at_address,Time_employed,Time_bank,Home_Expn,Balance,F,M,owner,rent,...,military,private_s,retired,self_empl,student,unemploye,given,oth_inst_,f,t
0,50.750000,0.585,0,0,145,0,0,1,1,0,...,0,0,0,0,0,1,1,0,1,0
1,19.670000,10.000,0,0,140,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,1
2,52.830002,15.000,5,14,0,2200,1,0,1,0,...,0,1,0,0,0,0,1,0,1,0
3,22.670000,2.540,2,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,1,0
4,29.250000,13.000,0,0,228,0,0,1,1,0,...,0,0,0,0,0,0,1,0,1,0
5,16.080000,0.335,0,1,160,126,1,0,1,0,...,0,0,0,0,0,1,1,0,1,0
6,23.170000,11.125,0,1,100,0,0,1,1,0,...,0,0,0,0,0,0,1,0,1,0
7,27.580000,3.000,2,1,280,10,1,0,1,0,...,0,1,0,0,0,0,1,0,0,1
8,19.170000,5.415,0,0,80,484,1,0,1,0,...,0,0,0,0,0,0,1,0,1,0
9,27.250000,0.290,0,1,272,108,1,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=0)

In [61]:
dt = tree.DecisionTreeClassifier(max_depth=5)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)


0.5813953488372093

In [62]:
y_pred = dt.predict(X_test)
dt.score(X_test, y_test)
y_pred = dt.predict(X_test)


In [63]:

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[16,  6],
       [12,  9]], dtype=int64)

In [64]:
accuracy_score(y_test, y_pred)*100

58.139534883720934

In [65]:
#Randon Forest using ensemble model
from sklearn.ensemble import RandomForestClassifier

random = ensemble.RandomForestClassifier(n_estimators=100)
random.fit(X_train, y_train)
random.score(X_test, y_test)


0.6744186046511628

# Using Chefboost to avoid encoding

In [6]:
from chefboost import Chefboost as chef
import pandas as pd

In [7]:
df = pd.read_excel("loan.xlsx")


#regular decision tree



In [10]:
config = {'algorithm':'ID3'}
model =chef.fit(df,config)

[INFO]:  2 CPU cores will be allocated in parallel running
ID3  tree is going to be built...
-------------------------
finished in  31.1659255027771  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  99.76689976689977 % on  429  instances
Labels:  ['reject' 'accept']
Confusion matrix:  [[236, 0], [1, 192]]
Precision:  100.0 %, Recall:  99.5781 %, F1:  99.7886 %


In [11]:
#Random Forest
config = {'algorithm':'ID3','enableRandomForest':True, 'num_of_trees':5}
model =chef.fit(df,config)


[INFO]:  2 CPU cores will be allocated in parallel running
ID3  tree is going to be built...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.69s/it]


-------------------------
finished in  18.80479407310486  seconds
-------------------------
Evaluate  train set
-------------------------
Accuracy:  81.58508158508158 % on  429  instances
Labels:  ['reject' 'accept']
Confusion matrix:  [[206, 48], [31, 144]]
Precision:  81.1024 %, Recall:  86.9198 %, F1:  83.9104 %
